In [1]:
from torchsig.utils.visualize import (
    MaskClassVisualizer,
    mask_class_to_outline,
    complex_spectrogram_to_magnitude,
)
from torchsig.transforms.target_transforms import DescToMaskClass, DescToListTuple
from torchsig.transforms import Spectrogram, Normalize
from torchsig.utils.writer import DatasetCreator, DatasetLoader
from torchsig.datasets.wideband_sig53 import WidebandSig53
from torchsig.datasets.wideband import WidebandModulationsDataset
from torchsig.transforms.transforms import Compose
from torchsig.datasets import conf
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

import json
import os
import pickle
from copy import deepcopy
from typing import Any, Dict, List, Tuple

import numpy as np

from torchsig.utils.types import SignalCapture, SignalDescription

In [9]:

SIGMF_DTYPE_MAP: Dict[str, np.dtype] = {
    "cf64_le": np.dtype("<f8"),
    "cf64_be": np.dtype(">f8"),
    "cf32_le": np.dtype("<f4"),
    "cf32_be": np.dtype(">f4"),
    "ci32_le": np.dtype("<i4"),
    "ci32_be": np.dtype(">i4"),
    "ci16_le": np.dtype("<i2"),
    "ci16_be": np.dtype(">i2"),
    "ci8_le": np.dtype("<i1"),
    "ci8_be": np.dtype(">i1"),
    "cu32_le": np.dtype("<u4"),
    "cu32_be": np.dtype(">u4"),
    "cu16_le": np.dtype("<u2"),
    "cu16_be": np.dtype(">u2"),
    "cu8_le": np.dtype("<u1"),
    "cu8_be": np.dtype(">u1"),
    "rf64_le": np.dtype("<f8"),
    "rf64_be": np.dtype(">f8"),
    "rf32_le": np.dtype("<f4"),
    "rf32_be": np.dtype(">f4"),
    "ri32_le": np.dtype("<i4"),
    "ri32_be": np.dtype(">i4"),
    "ri16_le": np.dtype("<i2"),
    "ri16_be": np.dtype(">i2"),
    "ri8_le": np.dtype("<i1"),
    "ri8_be": np.dtype(">i1"),
    "ru32_le": np.dtype("<u4"),
    "ru32_be": np.dtype(">u4"),
    "ru16_le": np.dtype("<u2"),
    "ru16_be": np.dtype(">u2"),
    "ru8_le": np.dtype("<u1"),
    "ru8_be": np.dtype(">u1"),
}


In [33]:
def indexer_from_folders_sigmf(root: str) -> List[Tuple[Any, SignalCapture]]:
    """An indexer where classes are delineated by folders

    Notes:
        Assumes data is stored as follows:
            root/class_x/xxx.sigmf-data
            root/class_x/xxx.sigmf-meta

            root/class_y/yxx.sigmf-data
            root/class_y/yxx.sigmf-meta

    Args:
        root:

    Returns:
        index: tuple of target, meta-data pairs

    """
    # go through directories and find files
    non_empty_dirs = [d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]
    non_empty_dirs.append(root)
    #non_empty_dirs = [d for d in non_empty_dirs if os.listdir(os.path.join(root, d))]
    #print(non_empty_dirs)
    # Identify all files associated with each class
    index = []
    for dir_idx, dir_name in enumerate(non_empty_dirs):
        class_dir = os.path.join(root, dir_name)

        # Find files with sigmf-data at the end and make a list
        proper_sigmf_files = list(
            filter(
                lambda x: x.split(".")[-1] in {"sigmf-data"}
                and os.path.isfile(os.path.join(class_dir, x)),
                os.listdir(os.path.join(root, dir_name)),
            )
        )

        # Go through each file and create and index
        for f in proper_sigmf_files:
            index = index + _parse_sigmf_anotations(os.path.join(class_dir, f))

    return index

def _parse_sigmf_anotations(absolute_file_path: str) -> List[SignalCapture]:
    """
    Args:
        absolute_file_path: absolute file path of sigmf-meta file for which to create Captures

    Returns:
        signal_files:

    """
    
    meta_file_name = "{}{}".format(absolute_file_path.split("sigmf-data")[0], "sigmf-meta")
    meta = json.load(open(meta_file_name, "r"))
    item_type = SIGMF_DTYPE_MAP[meta["global"]["core:datatype"]]
    sample_size = item_type.itemsize * (2 if "c" in meta["global"]["core:datatype"] else 1)
    total_num_samples = os.path.getsize(absolute_file_path) // sample_size

    # It's quite common for there to be only a single "capture" in sigMF
    index = []
    if len(meta["captures"]) == 1:
        for annotation in meta["annotations"]:
            sample_start = annotation["core:sample_start"]
            sample_count = annotation["core:sample_count"]
            signal_description = SignalDescription(
                sample_rate=meta["global"]["core:sample_rate"],
            )
            signal_description.upper_frequency = annotation["core:freq_upper_edge"]
            signal_description.lower_frequency = annotation["core:freq_lower_edge"]    
            label = annotation["core:label"]
            comment = annotation["core:comment"]


        
            signal =  SignalCapture(
                    absolute_path=absolute_file_path,
                    num_bytes=sample_size * sample_count,
                    byte_offset=sample_size * sample_start,
                    item_type=item_type,
                    is_complex=True if "c" in meta["global"]["core:datatype"] else False,
                    signal_description=signal_description,
                )
            index.append((label, signal))
            #print(f"Signal {label}  {signal.num_bytes} {signal.byte_offset} {signal.item_type} {signal.is_complex} ")
    else:
        print("Not Clear how we should handle the annotations when there is more than one capture")
    # If there's more than one, we construct a list of captures
    return index



In [34]:
from torchsig.utils.types import SignalCapture, SignalData
import torchsig.utils.reader as reader
from typing import Any, Callable, List, Optional, Tuple, Union
import numpy as np
import torch
class SignalDataset(torch.utils.data.Dataset):
    """An abstract dataset class to be sub-classed by SignalDatasets

    Args:
        transform:
            Transforms to be applied to SignalData Objects

        target_transform:
            Transforms to be applied to dataset targets

    """

    def __init__(
        self,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        seed: Optional[int] = None,
    ) -> None:
        super(SignalDataset, self).__init__()
        self.random_generator = np.random.RandomState(seed)
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(
        self,
        index: int,
    ) -> Tuple[Union[SignalData, np.ndarray], Any]:
        raise NotImplementedError

    def __len__(self) -> int:
        raise NotImplementedError

class SigMFDataset(SignalDataset):
    """SignalFileDataset is meant to make a mappable (index-able) dataset from
    a set of files

    Args:
        root:
            Root file path to search recursively for files

        indexer:
            Using root, constructs an index of data/meta-data

        reader:
            Given a file path, produces an SignalData object

        index_filter:
            Given an index, remove certain elements

        *\\*kwargs:**
            Keyword arguments

    """

    def __init__(
        self,
        root: str,
        indexer: Callable[[str], List[Tuple[Any, SignalCapture]]],
        reader: Callable[[SignalCapture], SignalData],
        index_filter: Optional[Callable[[Tuple[Any, SignalCapture]], bool]] = None,
        **kwargs,
    ):
        super(SigMFDataset, self).__init__(**kwargs)
        self.reader = reader
        self.index = indexer(root)
        if index_filter:
            self.index = list(filter(index_filter, self.index))

    def __getitem__(self, item: int) -> Tuple[np.ndarray, Any]:  # type: ignore
        target = self.index[item][0]
        signal_data = self.reader(self.index[item][1])

        if self.transform:
            signal_data = self.transform(signal_data)

        if self.target_transform:
            target = self.target_transform(target)

        return signal_data.iq_data, target  # type: ignore

    def __len__(self) -> int:
        return len(self.index)





In [35]:
dataset = SigMFDataset( root=".", indexer=indexer_from_folders_sigmf, reader=reader.reader_from_sigmf, index_filter=None)

In [37]:
print(dataset[0])

(array([   0. +0.j,  -16.-28.j,  -14. +7.j, ..., -113.-62.j, -131.-88.j,
       -127.-53.j]), 'signal')
